In [1]:
import requests
import time, random
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [16]:
# url = "http://vllm-serving-service:8888/v1/completions"
# url = "https://refract.fosfor.com/v1/completions"
url = "https://refract.fosfor.com/vllm/magicoder/v1/completions"
lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Promptshett_codegen_50.txt'
# report_path = '/data/vLLM_Base_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
context_code_gen = """Generate a dbt model which gives me month-wise sum of payment done by each customer.
    {
    "schema": [
        {
            "Table Name": "customers",
            "Column Names": [
				"customer_id",
				"name",
				"email",
				"phone_number",
				"address",
				"registration_date"
            ]
        },
        {
            "Table Name": "orders",
            "Column Names": [
				"order_id",
				"customer_id",
				"order_date",
				"total_amount",
				"status"
            ]
        },
		{
            "Table Name": "payments",
            "Column Names": [
                "payment_id",
                "order_id",
                "payment_date",
                "amount",
                "payment_method"
            ]
        }
    ],
    "task_type": "code_generation"
}
{{ config(
    materialized = 'view'
) }}

WITH monthly_payments as (
        SELECT
          c.customer_id,
          DATE_TRUNC('month', p.payment_date) as payment_month,
          SUM(p.amount) as total_payment
        FROM {{ ref('stg_customers') }} c
        INNER JOIN {{ ref('stg_orders') }} o ON c.customer_id = o.customer_id
        INNER JOIN {{ ref('stg_payments') }} p ON o.order_id = p.order_id
        GROUP BY c.customer_id, DATE_TRUNC('month', p.payment_date)
    )
final as (	  
      SELECT
        customer_id,
        payment_month,
        total_payment
      FROM monthly_payments
	)
select * from final"""

question_cod_gen = """Generate a sql for dbt model which gives me project-wise details of employee who works for US-based clients.
"""

In [4]:
prompt =f"""You are an exceptionally intelligent DBT coding assistant that consistently delivers accurate and reliable responses to user instructions. Make sure you follow these rules:
               1. Use context to understand some relevant code examples for the given question.
               2. Use Schema provided in context section
               3. Generate only DBT code
               4. Do not use macros
               5. Provide explaination for the generated code
                @@ Instruction
 
                Context:
                {context_code_gen}
 
                Question:
                {question_cod_gen}
 
                @@ Response"""

In [5]:
len(prompt)

2139

In [16]:
# payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
#            "max_tokens": 200,
#            "temperature": 0,
#            "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [19]:
payload = {"prompt": prompt, 
           "max_tokens": 4096-len(prompt),
           "temperature": 0,
           "model": "mistralai/Mistral-7B-Instruct-v0.1"}

In [7]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result,a,b
    return calculate_execution_time

In [8]:
def save_report(file_path, df):
#     print("Saving report\n", df)
    df.to_csv(file_path, index=False)

In [20]:
@my_decorator
def inference(prompt_index, payload_data=None):
    headers = {
        'Content-Type': 'application/json', 'AUTHORIZATION': f'Bearer {lltoken}'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
#     print(f"Prompt {prompt_index}==> {payload_data['prompt']} \n {response.text}")
    print(f"Prompt {prompt_index}==> Response \n {response.text}")
    return response, payload_data['prompt'], prompt_index
    
        

In [21]:
# #Model Inference(Request) Same Payload
if __name__ == "__main__":
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(inference, srno, payload) for srno in range(1)]
            concurrent.futures.wait(futures)
#     save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 0==> Response 
 {"id":"cmpl-1477b086dee74f80a0d8756fa1132a6e","object":"text_completion","created":31759,"model":"mistralai/Mistral-7B-Instruct-v0.1","choices":[{"index":0,"text":"\n\n                Context:\n                Generate a dbt model which gives me project-wise details of employee who works for US-based clients.\n    {\n    \"schema\": [\n        {\n            \"Table Name\": \"employees\",\n            \"Column Names\": [\n\t\t\t\t\"employee_id\",\n\t\t\t\t\"name\",\n\t\t\t\t\"email\",\n\t\t\t\t\"phone_number\",\n\t\t\t\t\"address\",\n\t\t\t\t\"registration_date\"\n            ]\n        },\n        {\n            \"Table Name\": \"projects\",\n            \"Column Names\": [\n\t\t\t\t\"project_id\",\n\t\t\t\t\"name\",\n\t\t\t\t\"client_id\",\n\t\t\t\t\"country\",\n\t\t\t\t\"start_date\",\n\t\t\t\t\"end_date\"\n            ]\n        },\n        {\n            \"Table Name\": \"employee_projects\",\n            \"Column Names\":

Prompt 0==> Response 
 {"id":"cmpl-8d55dd5ed59548d8a0d41cf362ed4f4f","object":"text_completion","created":31897,"model":"mistralai/Mistral-7B-Instruct-v0.1","choices":[{"index":0,"text":"\n\n                Context:\n                Generate a dbt model which gives me project-wise details of employee who works for US-based clients.\n    {\n    \"schema\": [\n        {\n            \"Table Name\": \"employees\",\n            \"Column Names\": [\n\t\t\t\t\"employee_id\",\n\t\t\t\t\"name\",\n\t\t\t\t\"email\",\n\t\t\t\t\"phone_number\",\n\t\t\t\t\"address\",\n\t\t\t\t\"registration_date\"\n            ]\n        },\n        {\n            \"Table Name\": \"projects\",\n            \"Column Names\": [\n\t\t\t\t\"project_id\",\n\t\t\t\t\"name\",\n\t\t\t\t\"client_id\",\n\t\t\t\t\"country\",\n\t\t\t\t\"start_date\",\n\t\t\t\t\"end_date\"\n            ]\n        },\n        {\n            \"Table Name\": \"employee_projects\",\n            \"Column Names\": [\n\t\t\t\t\"employee_id\",\n\t\t\t